Prepare the data

In [3]:
import pandas as pd

df = pd.read_csv("barcelona_listings.csv").loc[:200]
df.head()

/var/folders/3w/wrrjgv8n3kb0mj3897nsngbm0000gn/T/ipykernel_95366/1846378519.py:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("barcelona_listings.csv").loc[:200]


,id,listing_url,name,summary,space,description,neighborhood_overview,access,picture_url,host_id,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,18666,https://www.airbnb.com/rooms/18666,Flat with Sunny Terrace,"Apartment located near the ""Plaza de las Glori...",Nice apartment situated on the penthouse floor...,"Apartment located near the ""Plaza de las Glori...",Apartment in Barcelona near to the Plaza de la...,Free Wifi - air conditioning. We will provide ...,https://a0.muscache.com/im/pictures/47f88bc6-6...,71615,...,10/10/15,80.0,10.0,10.0,2.0,10.0,10.0,8.0,f,0.02
1,18674,https://www.airbnb.com/rooms/18674,Huge flat for 8 people close to Sagrada Familia,110m2 apartment to rent in Barcelona. Located ...,Apartment with 110 m2 located in the 6th floor...,110m2 apartment to rent in Barcelona. Located ...,Apartment in Barcelona located in the heart of...,Free Wifi - air conditioning. We will provide ...,https://a0.muscache.com/im/pictures/13031453/4...,71615,...,2/7/19,87.0,9.0,9.0,10.0,10.0,9.0,8.0,t,0.20
2,21605,https://www.airbnb.com/rooms/21605,Nice and sunny duble room,"The flat is in Poblenou district, and the room...","L'apartament està al barri de Poblenou, i l'ha...","The flat is in Poblenou district, and the room...",Poblenou as one of the few areas that has grow...,The kitchen is fully equipped and can use the ...,https://a0.muscache.com/im/pictures/774ca73d-1...,82522,...,4/7/19,90.0,10.0,9.0,10.0,10.0,9.0,9.0,f,3.08
3,23197,https://www.airbnb.com/rooms/23197,FORUM DELUXE 5 MINS WALK CCIB CENTER & SEA!,I do not accept groups of young people under 2...,Elegant spacious apartment suitable for 6. Amp...,I do not accept groups of young people under 2...,Strategically located in the area of Parc del ...,You book the entire apartment for yourselves.,https://a0.muscache.com/im/pictures/738532/806...,90417,...,7/7/19,95.0,10.0,10.0,10.0,10.0,9.0,9.0,t,0.44
4,25786,https://www.airbnb.com/rooms/25786,NICE ROOM AVAILABLE IN THE HEART OF GRACIA,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,Room available for rent.- PEDRO PEREZ. Share...,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,Solo decir que a menudo ni salgo del barrio. M...,All access with respect. Kitchen facilities ne...,https://a0.muscache.com/im/pictures/6619f0c7-8...,108310,...,3/7/19,95.0,10.0,10.0,10.0,10.0,10.0,9.0,t,2.22


In [4]:
df.isna().value_counts()

id     listing_url  name   summary  space  description  neighborhood_overview  access  picture_url  host_id  host_url  host_name  host_since  host_response_time  host_response_rate  host_is_superhost  host_picture_url  host_neighbourhood  host_listings_count  host_verifications  host_has_profile_pic  host_identity_verified  street  neighbourhood  neighbourhood_cleansed  neighbourhood_group_cleansed  city   zipcode  country  latitude  longitude  is_location_exact  property_type  room_type  accommodates  bathrooms  bedrooms  beds   amenities  square_feet  price  cleaning_fee  minimum_nights  maximum_nights  has_availability  availability_30  availability_60  availability_90  availability_365  number_of_reviews  number_of_reviews_ltm  first_review  last_review  review_scores_rating  review_scores_accuracy  review_scores_cleanliness  review_scores_checkin  review_scores_communication  review_scores_location  review_scores_value  instant_bookable  reviews_per_month
False  False        False

In [5]:
df.fillna("None", inplace=True)

/var/folders/3w/wrrjgv8n3kb0mj3897nsngbm0000gn/T/ipykernel_95366/3777244692.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("None", inplace=True)


Convert the releveant filed to vector using BERT model

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
import numpy
print(numpy.__version__)

2.1.3


In [ ]:
df["DescriptionVector"] = df["description"].apply(lambda x: model.encode(x))
df["DescriptionVector"].head(5)

Converting Input to embedding and preparing the context

In [ ]:
input_keyword = "Spacious apartment for large families or group of friends, modernly equipped"
vector_of_input_keyword = model.encode(input_keyword)
context = df["description"].iloc[0] +  "\n"   + df["description"].iloc[1] +  "\n"   + df["description"].iloc[2] +  "\n"   + df["description"].iloc[3] +  "\n"   + df["description"].iloc[4] +  "\n"
print(context)

Preraping Inference Client to interact with LLM

In [ ]:
from huggingface_hub import InferenceClient


import json
client = InferenceClient(
    provider="nebius",
    model="NousResearch/Hermes-4-405B",
    api_key="HF_TOKEN",
)
client

In [ ]:
question = "What temperature should I set my thermostat on vacation in the winter so that pipes dont freeze?"
question2 = "If you drop a soap on the floor,is the soap dirty or the floor clean?"
question  

Querying LLM using Hugging Face Inference Client

In [ ]:
completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user","content": question2
        }
    ],
)
completion.choices[0].message.content

Prompt before prompt example

In [ ]:
completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {"role": "system","content": "You are a helpful assistant that helps people find information.Please answer as if you are talking to a 10 year old."},
        {"role": "user","content": question2}
       
    ],
)
completion.choices[0].message.content

In [ ]:
completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {"role": "system","content": "You are a helpful assistant that helps people find information.Please answer in poem form."},
        {"role": "user","content": question2}
       
    ],
)
print(completion.choices[0].message.content)

RAG Demo : Providing Context to LLM to better assist user

In [ ]:
response = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {"role": "system", "content": "You are a helpful assistant who is helping airbnb system to respond to user questions. When you answer the question, use the first person (e.g. We) to refer to AirBNB."},
        {"role": "user", "content": input_keyword},
        {"role": "assistant", "content": f"Use this information as context to answer the user question: {context}. Please stick to the context and do not make up answers."}
    ]
)
print(input_keyword)
print(context)
response.choices[0].message.content

Calculating distance from available data to input query to find better match of data

In [ ]:
def fn(page_embedding):
    return numpy.dot(page_embedding, vector_of_input_keyword) 
df["distance"] = df["DescriptionVector"].apply(fn)
df.head()

Sorting available data to provide more releveant data to LLM

In [ ]:
df.sort_values(by="distance", ascending=False, inplace=True)
df

Preparing context based on sorted data

In [ ]:
context2 = df["description"].iloc[0] +  "\n"   + df["description"].iloc[1] +  "\n"   + df["description"].iloc[2] +  "\n"   + df["description"].iloc[3] +  "\n"   + df["description"].iloc[4] +  "\n"
print(context2)

Querying LLM with sorted context and input query

In [ ]:
response2 = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {"role": "system", "content": "You are a helpful assistant who is helping airbnb system to respond to user questions. When you answer the question, use the first person (e.g. We) to refer to AirBNB."},
        {"role": "user", "content": input_keyword},
        {"role": "assistant", "content": f"Use this information as context to answer the user question: {context2}. Please stick to the context and do not make up answers."}
    ]
)
print(input_keyword)
print(context2)
response2.choices[0].message.content